In [1]:
import torch
import torch.nn as nn
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import test_utils as t
import utils as u

In [3]:
input_width = 10
input_channels = 2
n_outputs = 5
dataset_size = 50
batch_size = 5

In [4]:
conv_model = t.TestConv()

In [5]:
modules_to_measure = {'conv.weight': {'input_shape': [input_width, input_width]},
                      'conv.bias': {'input_shape': [input_width, input_width]},
                      'fc.weight': {},
                      'fc.bias': {}}

In [6]:
test_data = torch.randn([dataset_size, input_channels, input_width, input_width])
test_labels = torch.randint(low=0,
                            high=n_outputs,
                            size=[dataset_size])

test_dataset = list(zip(test_data, test_labels))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)
grad_test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1)

In [7]:
outputs_to_measure = ['logit_sum', 'correct_logit', 'logit_margin', 'highest_incorrect_logit', 
                      'accuracy', 'cross_entropy']

In [8]:
measured_cumulants = u.measure_on_dataset(outputs_to_measure,
                                          conv_model,
                                          test_loader,
                                          torch.device('cpu'),
                                          conv_model.parameters())

In [9]:
measured_cumulants

{'logit_sum': [tensor(-0.0501), tensor(0.6898)],
 'correct_logit': [tensor(-0.0219), tensor(0.0997)],
 'logit_margin': [tensor(-0.3701), tensor(0.1615)],
 'highest_incorrect_logit': [tensor(0.3482), tensor(0.0600)],
 'accuracy': [tensor(0.1600), tensor(0.1344)],
 'cross_entropy': [tensor(1.6683), tensor(0.0835)]}

In [10]:
characteristics_to_measure = ['L2Norm', 'L1Norm', 'ApproxSpectralNorm', 'SpectralNorm', 'LinftyNorm', 'L2toInit']

In [11]:
measured_characteristics = u.measure_model_characteristics(characteristics_to_measure,
                                                          conv_model,
                                                          modules_to_measure,
                                                          conv_model.named_parameters())

In [12]:
conv_model.conv.weight.data.shape

torch.Size([4, 2, 3, 3])

In [13]:
measured_characteristics

{'conv.weight': {'L2Norm': 1.1872849464416504,
  'L1Norm': 8.544154167175293,
  'ApproxSpectralNorm': 0.7710604,
  'SpectralNorm': 1.50642414629988,
  'LinftyNorm': 0.2350132316350937,
  'L2toInit': 0.0},
 'conv.bias': {'L2Norm': 0.35680702328681946,
  'L1Norm': 0.7095252275466919,
  'ApproxSpectralNorm': 0.35680702328681946,
  'SpectralNorm': 0.35680702328681946,
  'LinftyNorm': 0.1998612880706787,
  'L2toInit': 0.0},
 'fc.weight': {'L2Norm': 1.3074249029159546,
  'L1Norm': 50.726802825927734,
  'ApproxSpectralNorm': 0.6235384345054626,
  'SpectralNorm': 0.6235384345054626,
  'LinftyNorm': 0.04998590424656868,
  'L2toInit': 0.0},
 'fc.bias': {'L2Norm': 0.06567399203777313,
  'L1Norm': 0.1165504977107048,
  'ApproxSpectralNorm': 0.06567399203777313,
  'SpectralNorm': 0.06567399203777313,
  'LinftyNorm': 0.043970268219709396,
  'L2toInit': 0.0}}

In [14]:
loss_fn = nn.CrossEntropyLoss(reduction='mean')

for step, (data, targets) in enumerate(grad_test_loader):
    u.zero_model_gradients(conv_model)

    outputs = conv_model(data)
    loss = loss_fn(outputs, targets)
    
    loss.backward()

In [13]:
measure_characteristics_internal = u.measure_characteristic_on_dataset(characteristics_to_measure,
                                                                       conv_model,
                                                                       grad_test_loader,
                                                                       torch.device('cpu'),
                                                                       modules_to_measure,
                                                                       list(conv_model.named_parameters()))